Let's import 'italian_names.txt' and create a list of names from it

In [2]:
words = open('italian_names.txt','r').read().splitlines()

In [3]:
type(len(w) for w in words)

generator

In [4]:
min(len(w) for w in words)

2

In [5]:
max(len(w) for w in words)

27

Let's build a **bigram**

In [17]:
b = {}
for w in words:
    chs = ['<S>'] + list(w) + ['<E>'] # add the start and the end character to each name
    for ch1, ch2 in zip(chs,chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1 # inizialize every new bigram to 0 occurrences

In [19]:
sorted(b.items(), key = lambda kv: -kv[1])

[(('o', '<E>'), 4235),
 (('a', '<E>'), 3829),
 (('i', 'n'), 1936),
 (('a', 'n'), 1500),
 (('r', 'i'), 1483),
 (('n', 'a'), 1430),
 (('i', 'o'), 1381),
 (('i', 'a'), 1344),
 (('n', 'o'), 1269),
 (('l', 'i'), 1262),
 (('e', 'r'), 1150),
 (('<S>', 'a'), 1095),
 (('e', 'l'), 1070),
 (('a', 'r'), 957),
 (('o', 'r'), 920),
 (('<S>', 'f'), 887),
 (('a', 'l'), 856),
 (('<S>', 'o'), 789),
 (('d', 'o'), 754),
 (('i', 'l'), 712),
 (('r', 'o'), 705),
 (('e', 'n'), 700),
 (('r', 'a'), 687),
 (('l', 'a'), 682),
 (('m', 'a'), 664),
 (('<S>', 'g'), 658),
 (('n', 'i'), 652),
 (('<S>', 'e'), 629),
 (('e', '<E>'), 626),
 (('<S>', 'r'), 626),
 (('<S>', 'm'), 598),
 (('o', 'n'), 591),
 (('r', 'e'), 584),
 (('t', 'a'), 578),
 (('d', 'a'), 565),
 (('t', 'o'), 560),
 (('d', 'i'), 552),
 (('o', 'l'), 547),
 (('d', 'e'), 540),
 (('l', 'l'), 537),
 (('m', 'i'), 533),
 (('s', 'i'), 531),
 (('<S>', 'c'), 520),
 (('l', 'e'), 510),
 (('g', 'i'), 497),
 (('i', 's'), 482),
 (('l', 'o'), 475),
 (('<S>', 'l'), 447),
 ((

It's more convinient for us to store this information in a **2-D array**: the rows are going to be the first character of each bigram and the columns are going to be the second character; each entry in this array will tell us how often a second character follows a first character inb the dataset. We will use a *tensor* from **Pytorch**.

In [20]:
import torch

ModuleNotFoundError: No module named 'torch'